In [1]:
import pickle
test1 = pickle.load(open('./pickled_objects/test1.p','rb'))
probssentences = pickle.load(open('./pickled_objects/probssentences.p', 'rb'))

In [21]:
#last sentence only
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0][-1]
    if probssentences[0][inx] > probssentences[1][inx]:
        preds.append(0)
    else:
        preds.append(1)
    

In [26]:
#standard averaging
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    if np.mean(probssentences[0][inx]) > np.mean(probssentences[1][inx]):
        preds.append(0)
    else:
        preds.append(1)


In [79]:
#max
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    if np.max(probssentences[0][inx]) > np.max(probssentences[1][inx]):
        preds.append(0)
    else:
        preds.append(1)


In [77]:
#apply sigmoidal function
import numpy as np
preds = []
k=10
for doc in [0]:#np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    scores0 = probssentences[0][inx].astype(float)-0.5
    scores0 = 1./(1+np.exp(-k*scores0))
    scores1 = probssentences[1][inx].astype(float)-0.5
    scores1 = 1./(1+np.exp(-k*scores1))
    if np.mean(scores0) > 0.5:
        preds.append(0)
    else:
        preds.append(1)

In [78]:
scores0

0    0.728297
1    0.007241
2    0.212901
3    0.366604
4    0.993279
Name: 0, dtype: float64

In [60]:
scores1

0    0.475370
1    0.620596
2    0.532642
3    0.513667
4    0.377639
Name: 1, dtype: float64

In [82]:
features = np.zeros((np.unique(probssentences["doc"]).shape[0],4))
features.shape

(12500, 4)

In [80]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], preds)

0.847807030176


## Decision Tree to see rule for selecting methods

In [83]:
import numpy as np
features = np.zeros((np.unique(probssentences["doc"]).shape[0],4))
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    features[doc,0] = np.mean(probssentences["doc"][inx])
    features[doc,1] = np.min(probssentences["doc"][inx])
    features[doc,2] = np.max(probssentences["doc"][inx])
    features[doc,3] = np.std(probssentences["doc"][inx])
    features[doc,4] = 

In [84]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features, test1["sentiment"])

In [93]:
clf.feature_importances_

array([ 0.3460855 ,  0.32288789,  0.33102661,  0.        ])

In [92]:
from IPython.display import Image  
from sklearn.externals.six import StringIO 
import pydot 
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data,  
                         #feature_names=iris.feature_names,  
                         #class_names=[0,1],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

KeyboardInterrupt: 